<a href="https://colab.research.google.com/github/albert119/newpr1/blob/main/recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Импорт данных и подготовка датасета

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings



from sklearn import preprocessing
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

warnings.filterwarnings('ignore')
%matplotlib inline



In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "<username>" 
os.environ['KAGGLE_KEY'] = "<key>"

In [ ]:
week16 = pd.read_csv('/content/drive/MyDrive/dip/csv/16.csv', usecols = [0,2,4,6,12, 9, 11] )

week17 = pd.read_csv('/content/drive/MyDrive/dip/csv/17.csv', usecols = [0,2,4,6,12, 9, 11] )

week18 = pd.read_csv('/content/drive/MyDrive/dip/csv/18.csv', usecols = [0,2,4,6,12, 9, 11])

week19 = pd.read_csv('/content/drive/MyDrive/dip/csv/19.csv', usecols = [0,2,4,6,12, 9, 11])

week20 = pd.read_csv('/content/drive/MyDrive/dip/csv/20.csv', usecols = [0,2,4,6,12, 9, 11])

week21 = pd.read_csv('/content/drive/MyDrive/dip/csv/21.csv', usecols = [0,2,4,6,12, 9, 11])

week22 = pd.read_csv('/content/drive/MyDrive/dip/csv/22.csv', usecols = [0,2,4,6,12, 9, 11])

week23 = pd.read_csv('/content/drive/MyDrive/dip/csv/23.csv', usecols = [0,2,4,6,12, 9, 11])

week24 = pd.read_csv('/content/drive/MyDrive/dip/csv/24.csv', usecols = [0,2,4,6,12, 9, 11])

In [ ]:
dataset = pd.concat([week16, week17, week18, week19, week20, week21, week22, week23, week24], ignore_index=True)

In [ ]:
week24

In [ ]:
dataset.rename(columns = {'Код товара':'Код_товара'}, inplace = True)
dataset.rename(columns = {'Закупочная цена магазина':'цена_зак'}, inplace= True)
dataset.rename(columns = {'Текущий тариф':'тариф'}, inplace= True)



In [ ]:
dataset['цена_зак'] = dataset['цена_зак'].replace(",",".", regex=True).astype(float)
dataset['тариф'] = dataset['тариф'].replace(",",".", regex=True).astype(float)

In [ ]:
  dataset['Рейтинг'] = ( dataset["тариф"].apply(lambda x: float(x))) - dataset["цена_зак"].apply(lambda x: float(x))

  dataset

In [ ]:
dataset['Рейтинг'] = preprocessing.normalize(dataset[['Рейтинг']], axis = 0 )
dataset

### Создание модели

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
n_weeks = len(dataset.Период.unique())
n_weeks

9

In [ ]:
n_items = len(dataset.Код_товара.unique())
n_items

909

In [ ]:
train

In [ ]:
items_input = Input(shape=[1], name="items-Input")
items_embedding = Embedding(10000000, 5, name="items-Embedding")(items_input)
items_vec = Flatten(name="Flatten-items")(items_embedding)

# creating user embedding path
weeks_input = Input(shape=[1], name="weeks-Input")
weeks_embedding = Embedding(30, 5, name="weeks-Embedding")(weeks_input)
weeks_vec = Flatten(name="Flatten-weeks")(weeks_embedding)

# performing dot product and creating model
prod = Dot(name="Dot-Product", axes=1)([items_vec, weeks_vec])
model = Model([weeks_input, items_input], prod)
model.compile('adam', 'mean_squared_error')

In [ ]:
from keras.models import load_model

if os.path.exists('regression_model.h5'):
    model = load_model('regression_model.h5')
else:
    history = model.fit([train.Период, train.Код_товара], train.Рейтинг,  epochs=5, verbose=1)
    model.save('regression_model.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

In [ ]:
model.evaluate([test.Период, test.Код_товара], test.Рейтинг)

7/7 [==============================] - 0s 2ms/step - loss: 8.0586e-04


0.000805857649538666

In [ ]:
predictions = model.predict([test.Период.head(10), test.Код_товара.head(10)])

[print(predictions[i], test.Рейтинг.iloc[i]) for i in range(0,10)]

### network

In [ ]:
# creating book embedding path
items_input = Input(shape=[1], name="items-Input")
items_embedding = Embedding(10000000, 5, name="items-Embedding")(items_input)
items_vec = Flatten(name="items-Books")(items_embedding)

# creating user embedding path
weeks_input = Input(shape=[1], name="weeks-Input")
weeks_embedding = Embedding(30, 5, name="weeks-Embedding")(weeks_input)
weeks_vec = Flatten(name="weeks-Users")(weeks_embedding)

# concatenate features
conc = Concatenate()([items_vec, weeks_vec])

# add fully-connected-layers
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

# Create model and compile it
model2 = Model([weeks_input, items_input], out)
model2.compile('adam', 'mean_squared_error')

In [ ]:
from keras.models import load_model

if os.path.exists('regression_model3.h5'):
    model2 = load_model('regression_model3.h5')
else:
    history = model2.fit([train.Период, train.Код_товара], train.Рейтинг, epochs=5, verbose=1)
    model2.save('regression_model3.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

In [ ]:
model2.evaluate([test.Период, test.Код_товара], test.Рейтинг)

In [ ]:
predictions = model2.predict([test.Период.head(10), test.Код_товара.head(10)])

[print(predictions[i], test.Рейтинг.iloc[i]) for i in range(0,10)]

### recomend

In [ ]:
items_data = np.array(list(set(dataset.Код_товара)))
items_data[:5]

array([233476, 907283, 907286, 747544, 540700])

In [ ]:

weeks_data = np.array([1 for i in range(5)])
weeks_data[:5]

array([1, 1, 1, 1, 1])

### win